In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_Data/", one_hot=True)

Extracting MNIST_Data/train-images-idx3-ubyte.gz
Extracting MNIST_Data/train-labels-idx1-ubyte.gz
Extracting MNIST_Data/t10k-images-idx3-ubyte.gz
Extracting MNIST_Data/t10k-labels-idx1-ubyte.gz


# Making the TensorFlow based CNN Complete Model

## Making the Helper Functions

In [12]:
def init_weights(shape):
    init_random = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random)

In [13]:
def init_bias(shape):
    init_bias_vals = tf.constant(1.0, shape = shape)
    return tf.Variable(init_bias_vals)

## Defining the Convolution functions

In [14]:
def Conv2D(x,W):
    return tf.nn.conv2d(input=x,filter=W,strides=(1,1,1,1),padding="SAME", use_cudnn_on_gpu=True)

In [15]:
def Pool2D(x):
    return tf.nn.max_pool(x,ksize=(1,2,2,1), strides=(1,2,2,1), padding="SAME")

## Making the Convolutional Layers

In [16]:
def Convolutional_layer(input_x, shape):
    W = init_weights(shape)
    c = init_bias([shape[3]])
    return tf.nn.relu(Conv2D(input_x,W) + c)

## Defining the Fully-Connected Layer

In [19]:
def full_layer(input_x, size):
    input_size = int(input_x.get_shape()[1])
    W = init_weights([input_size,size])
    c = init_bias([size])
    return tf.add(tf.matmul(input_x,W),c)

## Making the Final layers

In [20]:
# Making the placeholders
x = tf.placeholder(tf.float32, shape=[None,784])
y_true = tf.placeholder(tf.float32, shape=[None,10])


x_image = tf.reshape(x,[-1,28,28,1])

conv_1 = Convolutional_layer(x_image,shape=[5,5,1,32])
conv_1_pooling = Pool2D(conv_1)

conv_2 = Convolutional_layer(conv_1_pooling,shape = [5,5,32,64])
conv_2_pooling = Pool2D(conv_2)

# Prepare to feed the output of convolution layers to Fully-connected ANN architecture
conv_flatten_layer = tf.reshape(conv_2_pooling,shape=(-1,7*7*64))
full_layer_one = tf.nn.relu(full_layer(conv_flatten_layer,size = 1024))


In [21]:
# Dropout Layer
hold_prob = tf.placeholder(tf.float32)
full_layer_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

y_pred = full_layer(full_layer_dropout,size=10)

# Making the Loss Functions and initialization of the Model

In [22]:
# Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

# trainer
train = optimizer.minimize(cross_entropy)

In [23]:
# Running the session
init = tf.global_variables_initializer()

In [24]:
with tf.Session() as sess:
    sess.run(init)
    steps = 1000
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(44)
        sess.run(train, feed_dict = { y_true:batch_y, x:batch_x, hold_prob:0.5})
        
        if i%100==0:
            print("On Step: {}".format(i))
            print("Acuuracy is : ")
            
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,dtype=tf.float32))
            
            print(sess.run(acc, feed_dict={y_true:mnist.test.labels, x:mnist.test.images, hold_prob:1.0}))
            print("\n")

On Step: 0
Acuuracy is : 
0.0892


On Step: 100
Acuuracy is : 
0.731


On Step: 200
Acuuracy is : 
0.8725


On Step: 300
Acuuracy is : 
0.8904


On Step: 400
Acuuracy is : 
0.9067


On Step: 500
Acuuracy is : 
0.9245


On Step: 600
Acuuracy is : 
0.9226


On Step: 700
Acuuracy is : 
0.9308


On Step: 800
Acuuracy is : 
0.927


On Step: 900
Acuuracy is : 
0.9371


